<a href="https://colab.research.google.com/github/yashigautam30/yashigautam-/blob/main/Chap_1_cnn_for_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## STEP 1: SELECT RIGHT VERSION OF TENSORFLOW

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## STEP 2: CLONE GITHUB PROJECT (FOR EASY ACCESS TO DATASET)

In [19]:
!git clone https://github.com/mesushan/CNN-for-image-Classification.git

fatal: destination path 'CNN-for-image-Classification' already exists and is not an empty directory.


In [20]:
! ls

CNN-for-image-Classification  sample_data


In [21]:
import tensorflow as tf

In [22]:
# Initialising the CNN
model = tf.keras.models.Sequential()

## Step 3: ADDING CONVOLUTION LAYER

In [23]:
# 32 feature detectors with 3*3 dimensions so the convolution layer compose of 32 feature maps
# 128 by 128 dimensions with colored image(3 channels)  (tensorflow backend)
input_size = (128, 128)
model.add(tf.keras.layers.Convolution2D(32, 3, 3, input_shape = (*input_size, 3), activation = 'relu'))

## STEP 4: ADDING POOLING LAYER


In [24]:
# reduce the size of feature maps and therefore reduce the number of nodes in the future fully connected layer (reduce time complexity, less compute intense without losing the performace). 2 by 2 deminsion is the recommended option
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

## STEP 5: ADDING SECOND CONVOLUTION LAYER WITH POOLIING

In [25]:
model.add(tf.keras.layers.Convolution2D(32, 3, 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))


## STEP 6: ADDING FLATTENING LAYER

In [26]:
# flatten all the feature maps in the pooling layer into single vector
model.add(tf.keras.layers.Flatten())

## STEP 7: ADDING A FULLY CONNECTED LAYER

In [27]:
# making classic ann which compose of fully connected layers
# number of nodes in hidden layer (output_dim) (common practice is to take the power of 2)
model.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## STEP 8: COMPILING THE MODEL

In [28]:
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## STEP 9: FITTING THE CNN TO THE IMAGES

In [29]:
# image augmentation technique to enrich our dataset(training set) without adding more images so get good performance  results with little or no overfitting even with the small amount of images
# used from keras documentation (flow_from_directory method)

from keras.preprocessing.image import ImageDataGenerator
batch_size = 32
# image augmentation part
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# create training set
# wanna get higher accuracy -> inccrease target_size
training_set = train_datagen.flow_from_directory('/content/CNN-for-image-Classification/dataset/training_set',
                                                 target_size = input_size,
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')

# create test set
# wanna get higher accuracy -> inccrease target_size
test_set = test_datagen.flow_from_directory('/content/CNN-for-image-Classification/dataset/test_set',
                                            target_size = input_size,
                                            batch_size = batch_size,
                                            class_mode = 'binary')

# fit the cnn model to the trainig set and testing it on the test set
model.fit(training_set,
          steps_per_epoch = 8000/batch_size,
          epochs = 3,
          validation_data = test_set,
          validation_steps = 2000/batch_size)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/3
250/250 [==============================] - 48s 177ms/step - loss: 0.6872 - accuracy: 0.5452 - val_loss: 0.6704 - val_accuracy: 0.5610
Epoch 2/3
250/250 [==============================] - 42s 167ms/step - loss: 0.6617 - accuracy: 0.6105 - val_loss: 0.6345 - val_accuracy: 0.6490
Epoch 3/3
250/250 [==============================] - 42s 169ms/step - loss: 0.6350 - accuracy: 0.6355 - val_loss: 0.6103 - val_accuracy: 0.6720


## STEP 10: MAKING NEW *PREDICTIONS*

In [30]:
import numpy as np
from keras.preprocessing import image

In [31]:
test_image = image.load_img('/content/CNN-for-image-Classification/dataset/single_prediction/cat_or_dog_4.jpg', target_size= input_size)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

1/1 [==============================] - 0s 155ms/step


In [32]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [33]:
if result [0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [34]:
prediction

'dog'

### correct prediction made :)

#### The model shows the accuracy of 79.34 percent for the training set and 79.55 for the validation set. Can be improved with further parameter tuning if needed. ex. increasing the epochs, adding more convolution layer, changing input size of image, etc.